In [ ]:
!pip install albumentations
!pip install torchmetrics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import cv2
import cv2 as cv
import os
import pathlib
import sys
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torchmetrics
from Dataloaders import supervised_loader, test_loader, supervised_train_transform, test_transform

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

model = model.to(DEVICE)


In [4]:
LEARNING_RATE = 3e-5
BATCH_SIZE = 8
NUM_EPOCHS = 50
NUM_WORKERS = 2
IMAGE_HEIGHT = 736 
IMAGE_WIDTH = 1280 
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "segmentation_data/train_imgs"
TRAIN_MASK_DIR = "segmentation_data/train_masks"
VAL_IMG_DIR = "segmentation_data/val_imgs"
VAL_MASK_DIR = "segmentation_data/val_masks"

In [5]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

In [13]:
# val_data_size = 250
# ranset = set()
# while (len(ranset) <= val_data_size):
#     ranset.add(np.random.randint(0, 2001))

In [14]:
# import shutil
# import os

# imlis = os.listdir("images")
# lablis = os.listdir("thims")

In [15]:
# for i in ranset:
#     shutil.move(f"images/{imlis[i]}", "val_im")
#     shutil.move(f"thims/{lablis[i]}", "val_lab")

In [6]:
## loading dataloaders

train_data_loader = supervised_loader(image_dir = TRAIN_IMG_DIR, mask_dir= TRAIN_MASK_DIR,
                                      transform=supervised_train_transform, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,
                                      shape = (IMAGE_WIDTH, IMAGE_HEIGHT))

val_data_loader = supervised_loader(image_dir = VAL_IMG_DIR, mask_dir= VAL_MASK_DIR,
                                      transform=test_transform, batch_size=BATCH_SIZE, shuffle=False, num_workers = 4,
                                      shape = (IMAGE_WIDTH, IMAGE_HEIGHT))

In [7]:
iou_fn = torchmetrics.classification.JaccardIndex(task = 'binary', threshold = 0., num_classes = 2)

epochs = (range(NUM_EPOCHS))

val_loss = []
train_loss = []

train_iou = []
val_iou = []

max_val_iou = 0
for epoch in epochs:
    
    total_val_loss = 0
    total_train_loss = 0
    total_train_iou = 0
    total_val_iou = 0
    
    ## training part

    for batch_idx, (data, targets) in enumerate(train_data_loader):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        iou_train = iou_fn(predictions.to("cpu"), ((targets > 0)*1).to("cpu"))
        total_train_iou += iou_train.item()
        # loop.set_postfix(loss=loss.item(), iou_loss = iou_train_loss)
        total_train_loss += loss.item()
        print(f" Epoch - {epoch}, Step - {batch_idx+1}/{len(train_data_loader)}, Training_Loss - {round(total_train_loss/(batch_idx+1), 4)}, Train_mIOU - {round(total_train_iou/(batch_idx+1), 4)}", end="\r")
    train_loss.append(total_train_loss/(batch_idx+1))
    train_iou.append(total_train_iou/(batch_idx+1))

    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(val_data_loader):
            data = data.to(device=DEVICE)
            targets = targets.float().unsqueeze(1).to(device=DEVICE)

            predictions = model(data)
            loss = loss_fn(predictions, targets)
            total_validation_loss += loss
            iou_val = iou_fn(predictions.to("cpu"), ((targets > 0)*1).to("cpu"))
            total_val_iou += iou_val.item()
            # val_loop.set_postfix(val_loss=loss.item(), iou_loss = iou_val_loss)
    print(f" Epoch - {epoch}, Training_Loss - {round(total_train_loss/(batch_idx+1), 4)}, Train_mIOU - {round(total_train_iou/(batch_idx+1), 4)}, Training_Loss - {round(total_val_loss/(batch_idx+1), 4)}, val_mIOU - {round(total_val_iou/(batch_idx+1), 4)}")
    val_loss.append(total_val_loss/(batch_idx+1))
    val_iou.append(total_val_loss/(batch_idx+1))
    
    if total_val_loss/(batch_idx+1)>max_val_iou:
        print(f"Validation mIOU increased from {max_val_iou} to {total_val_loss/(batch_idx+1)}, Saving Model....")
        torch.save(model.state_dict(), "best_model.ckpt")
        max_val_iou = total_val_loss/(batch_idx+1)

[ WARN:0@45.248] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu2_mon--8_2022_6_20_3_4_44.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@45.779] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/kamalnayanbajaj_micu_mon--22_2022_6_20_21_47_17.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@46.477] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu3_mon--2_2022_6_8_16_2_30.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@47.170] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu2_mon--8_2022_6_20_18_5_13.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@47.867] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu2_mon--4_2022_6_20_6_3_58.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@48.553] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu2_mon--8_2022_5_18_6_6_53.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@49.267] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu3_mon--2_2022_5_18_22_2_47.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@49.963] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/hcgnagpur_icu_mon--1_2022_6_20_18_30_1.jpeg'): can't open/read file: check file path/integrity


[ WARN:0@50.660] global loadsave.cpp:244 findDecoder imread_('segmentation_data/train_imgs/medicakolkata_ccu3_mon--5_2022_5_18_3_3_0.jpeg'): can't open/read file: check file path/integrity


error: Caught error in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/srv/conda/envs/notebook/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/jovyan/PlanetaryComputerExamples/cloudphy/Dataloaders.py", line 23, in __getitem__
    image = cv2.resize(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB), self.shape)
cv2.error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



In [ ]:
plt.plot(train_loss_lis)

In [ ]:
plt.plot(train_loss_lis)

In [ ]:
plt.plot(val_loss_lis)

In [ ]:
plt.plot(val_iou)

In [ ]:
plt.plot(train_iou)

In [ ]:
torch.save(model.state_dict(), f"unet1750_epoch")